In [1]:
#importing packages
from keras.preprocessing.text import Tokenizer
import numpy as np
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding
from pickle import dump,load
from keras.callbacks import ModelCheckpoint
from keras import optimizers
from keras.models import load_model

Using TensorFlow backend.


In [2]:
#reading and processing the file
def read_file(path):
    with open(path) as f:
        stext = f.read().replace('\n', ' ')
    return stext

text = read_file('med_conv.txt').lower()
tokens = text.split(' ')

In [3]:
print(len(text),len(tokens))

339958 70558


In [4]:
#splitting the text into 5+1 words to predict the next word
train_len = 3+1
text_seq = []
for i in range(train_len, len(tokens)):
    seq = tokens[i-train_len:i]
    text_seq.append(seq)

sequences={}
cnt = 1
for i in range(len(tokens)):
    if tokens[i] not in sequences:
        sequences[tokens[i]] = cnt
        cnt += 1

In [5]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_seq)
sequences = tokenizer.texts_to_sequences(text_seq) 

#collecting info
vocab_size = len(tokenizer.word_counts)
n_seq = np.empty([len(sequences),train_len],dtype = 'int32')
for i in range(len(sequences)):
    n_seq[i] = sequences[i]


In [6]:
x_train = n_seq[:,:-1]
y_train = n_seq[:,-1]

y_train = to_categorical(y_train,num_classes=vocab_size+1)
seq_len = x_train.shape[1]

In [7]:
seq_len

3

In [8]:
def create_model(vocab_size, seq_len):
    model = Sequential()
    model.add(Embedding(vocab_size,seq_len,input_length=seq_len))
    model.add(LSTM(50, return_sequences=True))
    model.add(LSTM(50))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(vocab_size, activation='softmax'))
    optimizer = optimizers.adam(lr=0.001)
    
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    model.summary
    return model

In [9]:
model = create_model(vocab_size+1,seq_len)
filepath="weights-improvement-{epoch:02d}-bigger.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss',verbose=1,save_best_only=True, mode='min')
desired_callbacks = [checkpoint]

Instructions for updating:
Colocations handled automatically by placer.


In [10]:
#model.fit(x_train,y_train, epochs = 300, batch_size=128, callbacks=desired_callbacks)

In [11]:
#model.save('word_pred_Model4.h5')
model.load_weights('weights-improvement-299-bigger.hdf5')

In [12]:
dump(tokenizer,open('tokenizer_Model4','wb')) 

In [ ]:
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from pickle import load
import numpy as np
#model = load_model('word_pred_Model4.h5')
tokenizer = load(open('tokenizer_Model4','rb'))
seq_len = 3 
def gen_text(model, tokenizer, seq_len, seed_text, num_gen_words):
    output_text = []
    input_text = seed_text
    for i in range(num_gen_words):
        encoded_text = tokenizer.texts_to_sequences([input_text])[0]
        pad_encoded = pad_sequences([encoded_text], maxlen=seq_len,truncating='pre')
        pred_word_ind = model.predict_classes(pad_encoded,verbose=0)[0]
        (pred_word_ind)
        
        pred_word = tokenizer.index_word[pred_word_ind]
        input_text += ' '+pred_word
        output_text.append(pred_word)
    return ' '.join(output_text)

print('\n\n*** Enter exit_me to exit ')
while True:
    seed_text  = input('Enter the string: ')
    if seed_text.lower() == 'exit_me':
        break
    else:
        out = gen_text(model, tokenizer, seq_len=seq_len, seed_text=seed_text, num_gen_words=3)
        print('Output: '+seed_text+' '+out)



*** Enter exit_me to exit 
Enter the string: my knee is
Output: my knee is hurting so much.
Enter the string: i feel like
Output: i feel like i can hardly
Enter the string: i feel a
Output: i feel a sharp pain in
Enter the string: i have a
Output: i have a pain that radiates
Enter the string: my foot is
Output: my foot is hard coming could
Enter the string: i feel great
Output: i feel great pain in my
